Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Service Type

In [2]:
from gssutils import *
if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
    inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)
    tab = loadxlstabs(inputFile, sheetids='Table 3')[0]

In [4]:
observations = tab.excel_ref('B15').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)  


In [5]:
observations

{<F15 2482.0>, <G18 '*'>, <H15 1062.0>, <D20 91.8>, <C15 324.0>, <F18 '*'>, <B19 '-'>, <L20 79.6>, <G15 1712.0>, <M21 3.5>, <B20 98.9>, <L16 225.0>, <J17 2297.0>, <C20 99.1>, <M15 5969.0>, <E20 96.1>, <G19 '*'>, <H16 '*'>, <B16 '-'>, <E21 '*'>, <H20 76.3>, <F16 '*'>, <K18 45.0>, <D15 294.0>, <F17 2203.0>, <M18 211.0>, <B18 '*'>, <L18 51.0>, <E18 '*'>, <K16 27.0>, <K20 96.5>, <J16 165.0>, <D17 270.0>, <K21 2.2>, <I15 5256.0>, <M16 417.0>, <L21 3.8>, <I17 4656.0>, <K17 1964.0>, <J20 89.1>, <L15 1356.0>, <H21 '*'>, <J18 115.0>, <G16 '*'>, <B21 '*'>, <J15 2577.0>, <I18 '*'>, <L17 1080.0>, <L19 16.6>, <H17 810.0>, <K15 2036.0>, <K19 1.3>, <C17 321.0>, <E19 '-'>, <E17 685.0>, <F20 88.8>, <H18 '*'>, <B15 95.0>, <E15 713.0>, <H19 '*'>, <F21 '*'>, <I21 '*'>, <M19 7.0>, <B17 94.0>, <J21 4.5>, <E16 '-'>, <G17 1643.0>, <I20 88.6>, <F19 '*'>, <J19 6.4>, <M17 5341.0>, <I16 '*'>, <G21 '*'>, <G20 96.0>, <I19 '*'>, <M20 89.5>}

In [6]:
Service = tab.excel_ref('A15').expand(DOWN).is_not_blank()
Service

{<A18 'Mixed'>, <A17 'Non-residential '>, <A15 'Total'>, <A21 'Mixed (%)'>, <A19 'Residential (%)'>, <A20 'Non-residential (%)'>, <A16 'Residential'>}

In [7]:
Treatment = tab.excel_ref('B14').expand(RIGHT).is_not_blank()
Treatment

{<H14 'Drugs & Alcohol'>, <C14 'Drugs Only'>, <L14 'Drugs & Alcohol'>, <F14 'Alcohol Only'>, <G14 'Drugs Only'>, <J14 'Alcohol Only'>, <D14 'Drugs & Alcohol'>, <I14 'Total'>, <K14 'Drugs Only'>, <E14 'Total'>, <B14 'Alcohol Only'>}

In [8]:
age = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
age

{<J13 'Treatment Type'>, <B13 'Under 18 '>, <M13 'Overall Total'>, <F13 '18 and over'>}

In [9]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Category',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','Persons'),
            HDimConst('Period','2006-2016')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Treatment Type,Category,Age,Measure Type,Unit,Sex,Period
0,95,NaN,Alcohol Only,Total,Under 18,Count,People,Persons,2006-2016
1,324,NaN,Drugs Only,Total,Under 18,Count,People,Persons,2006-2016
2,294,NaN,Drugs & Alcohol,Total,Under 18,Count,People,Persons,2006-2016
3,713,NaN,Total,Total,Under 18,Count,People,Persons,2006-2016
4,2482,NaN,Alcohol Only,Total,18 and over,Count,People,Persons,2006-2016
5,1712,NaN,Drugs Only,Total,18 and over,Count,People,Persons,2006-2016
6,1062,NaN,Drugs & Alcohol,Total,18 and over,Count,People,Persons,2006-2016
7,5256,NaN,Total,Total,18 and over,Count,People,Persons,2006-2016
8,2577,NaN,Alcohol Only,Total,Treatment Type,Count,People,Persons,2006-2016
9,2036,NaN,Drugs Only,Total,Treatment Type,Count,People,Persons,2006-2016


In [12]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [13]:
new_table.dtypes

Value             object
DATAMARKER        object
Treatment Type    object
Category          object
Age               object
Measure Type      object
Unit              object
Sex               object
Period            object
dtype: object

In [14]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Category,Age,Measure Type,Unit,Sex,Period
71,,*,Total,Mixed (%),18 and over,Count,People,Persons,2006-2016
72,4.5,NaN,Alcohol Only,Mixed (%),Treatment Type,Count,People,Persons,2006-2016
73,2.2,NaN,Drugs Only,Mixed (%),Treatment Type,Count,People,Persons,2006-2016
74,3.8,NaN,Drugs & Alcohol,Mixed (%),Treatment Type,Count,People,Persons,2006-2016
75,3.5,NaN,None,Mixed (%),Overall Total,Count,People,Persons,2006-2016


In [15]:
new_table.count()

Value             76
DATAMARKER        24
Treatment Type    69
Category          76
Age               76
Measure Type      76
Unit              76
Sex               76
Period            76
dtype: int64

In [16]:
new_table = new_table[new_table['Value'] !=  0 ]

In [17]:
new_table = new_table[new_table['Value'] !=  '' ]

In [18]:
new_table.count()

Value             52
DATAMARKER         0
Treatment Type    45
Category          52
Age               52
Measure Type      52
Unit              52
Sex               52
Period            52
dtype: int64

In [19]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [20]:
new_table['Treatment Type'].fillna('Total', inplace = True)

In [21]:
new_table = new_table[['Period','Category','Age','Sex','Treatment Type','Measure Type','Value','Unit']]

In [22]:
new_table.head(5)

,Period,Category,Age,Sex,Treatment Type,Measure Type,Value,Unit
0,2006-2016,Total,Under 18,Persons,Alcohol Only,Count,95,People
1,2006-2016,Total,Under 18,Persons,Drugs Only,Count,324,People
2,2006-2016,Total,Under 18,Persons,Drugs & Alcohol,Count,294,People
3,2006-2016,Total,Under 18,Persons,Total,Count,713,People
4,2006-2016,Total,18 and over,Persons,Alcohol Only,Count,2482,People
